In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import os
from pyspark.sql.functions import count,col,when, collect_list,concat_ws,lit,monotonically_increasing_id,first
from pyspark.sql.dataframe import *
from pyspark.sql.types import TimestampType, StringType
from pyspark.sql import SparkSession, functions as F

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
import os

spark = SparkSession.builder.appName("IcebergNotebook").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

spark.sql('CREATE NAMESPACE IF NOT EXISTS lakehouse.nyc').show()
print(f"Namespace Iceberg créée : nyc")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/03 02:21:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/03 02:21:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
from pyspark.sql.types import DoubleType, FloatType, LongType, StructType,StructField, StringType
schema = StructType([
  StructField("vendor_id", LongType(), True),
  StructField("trip_id", LongType(), True),
  StructField("trip_distance", FloatType(), True),
  StructField("fare_amount", DoubleType(), True),
  StructField("store_and_fwd_flag", StringType(), True)
])

df = spark.createDataFrame([], schema)
df.writeTo("lakehouse.nyc.taxis").createOrReplace()

In [ ]:
schema = spark.table("lakehouse.nyc.taxis").schema
data = [
    (1, 1000371, 1.8, 15.32, "N"),
    (2, 1000372, 2.5, 22.15, "N"),
    (2, 1000373, 0.9, 9.01, "N"),
    (1, 1000374, 8.4, 42.13, "Y")
  ]
df = spark.createDataFrame(data, schema)
df.writeTo("lakehouse.nyc.taxis").append()

In [ ]:
df = spark.table("lakehouse.nyc.taxis").show()

+---------+-------+-------------+-----------+------------------+
|vendor_id|trip_id|trip_distance|fare_amount|store_and_fwd_flag|
+---------+-------+-------------+-----------+------------------+
|        1|1000371|          1.8|      15.32|                 N|
|        2|1000372|          2.5|      22.15|                 N|
|        2|1000373|          0.9|       9.01|                 N|
|        1|1000374|          8.4|      42.13|                 Y|
+---------+-------+-------------+-----------+------------------+



In [ ]:
spark.sql("SHOW TBLPROPERTIES lakehouse.nyc.taxis;").show()


+--------------------+--------------------+
|                 key|               value|
+--------------------+--------------------+
|          created-at|2025-12-03T02:22:...|
| current-snapshot-id| 7795437363922395310|
|              format|     iceberg/parquet|
|      format-version|                   2|
|write.parquet.com...|                zstd|
+--------------------+--------------------+

